# Importing and preparing supermarkets data

## Libraries and settings

In [1]:
# Libraries
import os
import fnmatch
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Get current working directory
print('Current working directory:', os.getcwd())

# Show .json files in the current working directory
flist = fnmatch.filter(os.listdir('.'), '*.json')
for i in flist:
    print(i)

Current working directory: c:\Users\diego\OneDrive\Documentos\GitHub\data_analytics_zhaw\Week_02
supermarkets.json


## Importing data

In [2]:
# Read the data to a pandas data frame
df1 = pd.read_json('supermarkets.json', encoding='utf-8')
df1.head(5)

,type,id,lat,lon,tags
0,node,33126515,47.155616,9.037915,"{'brand': 'Spar', 'brand:wikidata': 'Q610492',..."
1,node,36726161,47.226191,8.980329,"{'addr:city': 'Uznach', 'addr:housenumber': '2..."
2,node,39768209,47.225069,8.969981,"{'addr:city': 'Uznach', 'addr:postcode': '8730..."
3,node,39947904,47.376732,8.542161,"{'addr:city': 'Zürich', 'addr:country': 'CH', ..."
4,node,48932835,47.375020,8.522895,"{'addr:city': 'Zürich', 'addr:housenumber': '7..."


## Count number of rows and columns in the data frame

In [3]:
# Dimension (rows, columns)
print('Dimension:', df1.shape)

# Number of rows
print('Number of rows:', df1.shape[0])

# Number of columns
print('Number of columns:', df1.shape[1])

Dimension: (3392, 5)
Number of rows: 3392
Number of columns: 5


## Column 'tags' is a pandas Series with dictionaries -> change to data frame

In [4]:
# Type of the first item of column tags
print(type(df1.tags))
print(type(df1.tags[0]))

# Content of the first item of column tags
print(df1.tags[0].keys())

# Change to data frame
df2 = pd.DataFrame.from_records(df1.tags)
df2 = df2[['brand', 'shop', 'addr:city', 'addr:street', 'addr:housenumber', 'addr:postcode']]

# Rename selected columns
df2 = df2.rename(columns={'addr:city': 'city',
                          'addr:street':'street',
                          'addr:housenumber': 'housenumber',
                          'addr:postcode': 'postcode'})

# Show first records of data frame
df2.head()

<class 'pandas.core.series.Series'>
<class 'dict'>
dict_keys(['brand', 'brand:wikidata', 'brand:wikipedia', 'name', 'opening_hours', 'shop'])


,brand,shop,city,street,housenumber,postcode
0,Spar,supermarket,NaN,NaN,NaN,NaN
1,Migros,supermarket,Uznach,Zürcherstrasse,25,8730
2,Coop,supermarket,Uznach,NaN,NaN,8730
3,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001
4,Migros,supermarket,Zürich,Wengistrasse,7,8004


## Merge df1 and df2

In [5]:
# Merge df and df2
df = pd.merge(df1[['type', 'id', 'lat', 'lon']], 
              df2[['brand', 'shop', 'city', 'street', 'housenumber', 'postcode']],
              left_index=True, 
              right_index=True)
df.head(5)

,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730
2,node,39768209,47.225069,8.969981,Coop,supermarket,Uznach,NaN,NaN,8730
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004


## Count and identify the number of missing values (if any)

In [6]:
# Count missing values
print(pd.isna(df).sum())

# Identify rows with missing values, e.g.:
df.loc[pd.isna(df['city'])]

type              0
id                0
lat               0
lon               0
brand          1065
shop              0
city           1777
street         1608
housenumber    1680
postcode       1709
dtype: int64


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN
5,node,60271452,47.406671,9.305450,NaN,supermarket,NaN,NaN,NaN,NaN
6,node,70656485,47.491253,8.733981,NaN,supermarket,NaN,NaN,NaN,NaN
10,node,81321513,47.532917,9.066408,Landi,supermarket,NaN,NaN,NaN,NaN
13,node,95582038,47.050385,9.059214,NaN,supermarket,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3384,node,11083317088,46.862184,9.531169,Lidl,supermarket,NaN,NaN,NaN,NaN
3386,node,11098091830,46.205111,6.130174,Coop,supermarket,NaN,NaN,NaN,NaN
3387,node,11099817248,46.928691,7.561873,NaN,supermarket,NaN,NaN,NaN,NaN
3388,node,11103235832,46.166742,8.771970,Migros,supermarket,NaN,NaN,NaN,NaN


In [7]:
df['type'].unique()

array(['node'], dtype=object)

## Count and identify duplicated values (if any)

In [8]:
# Count duplicated values
print(df.duplicated().sum())

# Identify rows with duplicated values, e.g.:
df[df[['id']].duplicated()]

0


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode


## Get data types of all variables

In [9]:
# Get data types (note that in pandas, a string is referred to as 'object')
df.dtypes

type            object
id               int64
lat            float64
lon            float64
brand           object
shop            object
city            object
street          object
housenumber     object
postcode        object
dtype: object

Additional filters on supermarkets

In [10]:
df['brand'].unique()

array(['Spar', 'Migros', 'Coop', nan, 'ALDI', 'Landi', 'Pam', 'Aldi',
       'Volg', 'VOI Migros Partner', 'Denner', 'Lidl', 'Crai', 'VOI',
       'Migros-Outlet', 'gazi', 'Coop;wikidata=Q432564', 'Lidl Schweiz',
       'coop', 'Eurospar', 'Alnatura', 'Denner;Migros', '7-Eleven',
       'Aldi Suisse', 'Rolog Krinner Cotub Titan Serac Molok', 'Jumbo',
       'avec', 'Metzgerei Eichenberger', 'ZWIBOL', 'Head, Fischer'],
      dtype=object)

In [12]:
df_f = df.loc[(df['brand'] == 'Migros') & (df['city'] == 'Zürich')]
df_f.head(5)

,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004
11,node,83330862,47.344749,8.529981,Migros,supermarket,Zürich,Etzelstrasse,3,8038
16,node,119249170,47.375255,8.536107,Migros,supermarket,Zürich,Löwenstrasse,31-35,8001
50,node,262400822,47.364072,8.530945,Migros,supermarket,Zürich,Tessinerplatz,10,8002
71,node,267346993,47.385598,8.531471,Migros,supermarket,Zürich,Limmatstrasse,152,8005


In [13]:
df['city'].unique()

array([nan, 'Uznach', 'Zürich', 'Winterthur', 'Schötz', 'Mollis', 'Stäfa',
       'Benken SG', 'Lenzburg', 'Luzern', 'Frick', 'Pratteln',
       'Breil/Brigels', 'Kaltbrunn', 'Wildhaus', 'Weinfelden',
       'Altstätten', 'Wallisellen', 'Baden', 'Dättwil', 'Brugg', 'Horgen',
       'Basel', 'Uster', 'Morges', 'Lutry', 'Reinach BL', 'Laufen',
       'St. Gallen', 'Siebnen', 'Hochdorf', 'Nussbaumen AG', 'Schänis',
       'Bern', 'Richterswil', 'Sarnen', 'Lausanne', 'Breitenbach',
       'Villars-sur-Ollon', 'Kandersteg', 'Allschwil', 'Pully',
       'Yverdon-les-Bains', 'Schwyz', 'Stadel b. Niederglatt', 'Sulgen',
       'Thun', 'Langnau am Albis', 'Thalwil', 'Grindelwald', 'Fällanden',
       'Pfäffikon ZH', 'Wohlen', 'Bauma', 'Bassersdorf', 'Nürensdorf',
       'Herzogenbuchsee', 'Echallens', 'Neuenhof', 'Niederhasli',
       'Wädenswil', 'Port', 'Mettmenstetten', 'Münsingen', 'Emmenbrücke',
       'Russikon', 'Biel/Bienne', 'Genève', 'Geneva', 'Arlesheim',
       'Ehrendingen', 'Rotkr

In [27]:
df_g = df.loc[(df['brand'] == 'coop')]
df_g.head(5)

UndefinedVariableError: name 'Zurich' is not defined

### Save data to file

In [ ]:
df.to_csv('supermarkets_data_prepared.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')